In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain_anthropic import ChatAnthropic
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", google_api_key=os.getenv("GOOGLE_API_KEY")
)
import os

# Load documents
loader = WebBaseLoader("https://www.anthropic.com/claude")
documents = loader.load()

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", api_key=os.getenv("GOOGLE_API_KEY")
)

# Create vector store
vector_store = Chroma.from_documents(documents, embeddings)

# Initialize conversation memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize language model
llm = ChatAnthropic(
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-sonnet-20240229",
    temperature=0.2,
    max_tokens=1024,
)

# Create the chain
qa = ConversationalRetrievalChain.from_llm(
    llm, vector_store.as_retriever(), memory=memory
)

# Start the conversation
chat_history = []
query = "What is Anthropics lightest model?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

# Continue the conversation
query = "What The most intelliigent?"
chat_history = result["chat_history"]
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])